In [ ]:
import torch
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
import torch
from torch import optim
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#  Load Model and Tokenize
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/bart-base-cnn")
model_bart = BartForConditionalGeneration.from_pretrained("ainize/bart-base-cnn")


test_data = pd.read_csv("test_text.csv")

batch_size = 4  # choix de taille de batch appropriée
num_batches = (len(test_data) + batch_size - 1) // batch_size

generated_summaries = []

for i in range(num_batches):
    batch_data = test_data.iloc[i * batch_size: (i + 1) * batch_size]
    
    test_encodings = tokenizer(batch_data["text"].tolist(), padding="max_length", truncation=True, max_length=128, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model_bart.generate(input_ids=test_encodings["input_ids"], max_length=50, num_beams=4, early_stopping=True)
    
    decoded_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    generated_summaries.extend(decoded_summaries)

submission_df = pd.DataFrame({"ID": test_data["ID"], "titles": generated_summaries})

submission_df.to_csv("bart_cnn_not_trained.csv", index=False)
